In [ ]:
import torch
from torchvision import transforms, models, datasets
from PIL import Image
import cv2
import requests
import json
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Define the same transformations used for validation during training
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load class names from a JSON file (assuming this file was saved during training)
try:
    with open('class_names.json', 'r') as f:
        class_names = json.load(f)
except FileNotFoundError:
    print("Error: 'class_names.json' not found. Please ensure the class names file from training is in the same directory.")
    exit() # Exit if class names are not found

num_classes = len(class_names) # Get number of classes dynamically

In [ ]:
# Load the pre-trained EfficientNet-B3 model structure
model = models.efficientnet_b3(pretrained=False) # No pretrained weights here initially

In [ ]:
# Get the number of features in the last fully connected layer
num_ftrs = model.classifier[1].in_features

# Redefine the classifier sequential block to exactly match the structure used in train_model.py
model.classifier[1] = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(512, 128),
    torch.nn.BatchNorm1d(128),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(128, num_classes) # Set the final layer size directly using num_classes
)

# Load the trained model weights
model.load_state_dict(torch.load('best_model.pth', map_location=device)) # Load the best model saved during training
model = model.to(device)

In [ ]:
# Set the model to evaluation mode
model.eval()

In [ ]:
# Function to make a prediction on a single image file
def predict_image(image_path, model, transform, class_names, device):
    try:
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0) 
        image = image.to(device)

        with torch.no_grad():
            outputs = model(image)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted_class_idx = torch.max(outputs, 1)

        predicted_class_name = class_names[predicted_class_idx.item()]
        confidence = probabilities[0][predicted_class_idx.item()].item()

        return predicted_class_name, confidence

    except FileNotFoundError:
        return f"Error: Image file not found at {image_path}", None
    except Exception as e:
        return f"An error occurred during prediction: {e}", None

In [ ]:
# Example usage:
if __name__ == '__main__':
    # Replace 'path/to/your/image.jpg' with the actual path to a plant leaf image file
    image_file_path = 'test\AppleScab2.JPG' 

    predicted_class, confidence = predict_image(image_file_path, model, transform, class_names, device)

    if confidence is not None:
        print(f"Predicted Class: {predicted_class}")
        print(f"Confidence: {confidence:.4f}")

    else:
        print(predicted_class) # Print the error message

In [ ]:
# --- Test Dataset Loading and Evaluation ---
# Assuming your test data is in a folder named 'test'
test_dataset = datasets.ImageFolder('test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False) 

print(f"Number of test images: {len(test_dataset)}")

true_labels = []

predicted_labels = []

# Perform inference on the test dataset in batches
print("Performing inference on the test dataset...")
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

print("Inference complete. Calculating evaluation metrics...")

In [ ]:
# Calculate and print evaluation metrics
print("\nComprehensive Evaluation Report:")
print("Confusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

print("\nClassification Report:")
# Ensure class_names is correctly loaded for target_names
print(classification_report(true_labels, predicted_labels, target_names=class_names))